## CNN autoencoder

## Prepare data

In [1]:
from matplotlib import pyplot
import keras
import tensorflow as tf
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import RepeatVector
from keras.layers import GRU
from keras.layers import Conv3D
from keras.layers import MaxPool3D
from keras.utils import plot_model
from keras.callbacks import EarlyStopping
import numpy as np
from numpy import array

In [3]:
from data_processing import process
filename = 'gear_dataset.csv'
rows = 1000
data, IDs = process(filename,rows)
n_rows = data.shape[0] 
n_features = data.shape[1]
print(data.shape)


(989, 64)
0140


## fixa så att det är uppdelat på kanske $40\cdot64\cdot2$

In [61]:
# do cube with stream*64*ID

def convert_from_hex(hex,output_type): # converts the data in hex from hexadecimal to decimal or binary form
     out = np.zeros((hex.size))
     if output_type == 'dec':
        for x in range(hex.size):
            h_value = hex[x]
            out[x] = int(h_value,16)
     else:
        for x in range(hex.size):
            h_value = hex[x]
            binary[x] = bin(int(h_value, 16))[2:]

     return out


#data = convert_from_hex(Data,'dec')
IDs = IDs.reset_index(drop=True)

id = convert_from_hex(IDs,'dec') 

ID_matrix =  array([[id],]*data.shape[1]).transpose()
ID_matrix = np.squeeze(ID_matrix)
dataCube = np.dstack([data,ID_matrix])

## Build network

In [68]:
# channels?? 2d or 3d? 
n_timesteps = dataCube.shape[0]
n_features = dataCube.shape[1]
n_3rd = 2
n_channels = 1
input = Input(shape=(n_timesteps,n_features,n_3rd,n_channels))

l1 = Conv3D(filters = 16, kernel_size = (3, 3, 3), activation='relu', padding='same')(input)
l2 = Conv3D(filters = 16, kernel_size = (3, 3, 3), activation='relu', padding='same')(l1)